In [1]:
# Copyright (c) Microsoft. All rights reserved.
import logging
import json
import textwrap
import re

from dotenv import load_dotenv

import asyncio
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import kernel_function

"""
The following sample demonstrates how to create a chat completion agent that
answers questions about a sample menu using a Semantic Kernel Plugin. The Chat
Completion Service is passed directly via the ChatCompletionAgent constructor.
Additionally, the plugin is supplied via the constructor.
"""

# Load environment variables from .env file
load_dotenv()

# NOTE: This is all that is required to enable logging.
# Set the desired level to INFO, DEBUG, etc.
logging.basicConfig(level=logging.WARNING)

In [2]:
# Define a sample plugin for the sample
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"


# Simulate a conversation with the agent
USER_INPUTS = [
    "Hello",
    "What is the special soup?",
    "What does that cost?",
    "Thank you",
]

# 1. Create the agent
agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o-mini"),
    name="Host",
    instructions="Answer questions about the menu.",
    plugins=[MenuPlugin()],
)

# 2. Create a thread to hold the conversation
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: ChatHistoryAgentThread = None

for user_input in USER_INPUTS:
    print(f"# User: {user_input}")
    # 4. Invoke the agent for a response
    response = await agent.get_response(messages=user_input, thread=thread)
    print(f"# {response.name}: {response} ")
    thread = response.thread

# 4. Cleanup: Clear the thread
await thread.delete() if thread else None

"""
Sample output:
# User: Hello
# Host: Hello! How can I assist you today?
# User: What is the special soup?
# Host: The special soup is Clam Chowder.
# User: What does that cost?
# Host: The special soup, Clam Chowder, costs $9.99.
# User: Thank you
# Host: You're welcome! If you have any more questions, feel free to ask. Enjoy your day!
"""


# User: Hello
# Host: Hello! How can I assist you today? 
# User: What is the special soup?
# Host: The special soup is Clam Chowder. Would you like to know more about the menu or anything else? 
# User: What does that cost?
# Host: The Clam Chowder costs $9.99. If you have any more questions or need further assistance, feel free to ask! 
# User: Thank you
# Host: You're welcome! If you need anything else, just let me know. Enjoy your day! 


"\nSample output:\n# User: Hello\n# Host: Hello! How can I assist you today?\n# User: What is the special soup?\n# Host: The special soup is Clam Chowder.\n# User: What does that cost?\n# Host: The special soup, Clam Chowder, costs $9.99.\n# User: Thank you\n# Host: You're welcome! If you have any more questions, feel free to ask. Enjoy your day!\n"